graph db
conjuctive regaulr path queires poptimization


Cardinality estimation

https://15799.courses.cs.cmu.edu/spring2025/notes/07-joins1.pdf
https://15799.courses.cs.cmu.edu/spring2025/schedule.html
https://www.microsoft.com/en-us/research/publication/extensible-query-optimizers-in-practice/ book

starburst
volcano 
cascades

DPhyp
enumerate all
hypergraph



https://arxiv.org/pdf/2407.04823 Path-based Algebraic Foundations of Graph Query Languages
https://openproceedings.org/2025/conf/edbt/paper-234.pdf Path-based Algebraic Foundations of Graph Query Languages

https://dl.acm.org/doi/abs/10.1145/3725237 Minimizing Conjunctive Regular Path Queries

https://en.wikipedia.org/wiki/Regular_path_query
https://homepages.inf.ed.ac.uk/libkin/papers/icdt12b.pdf

networkx?
sqlite?

SQL/PCQ
GQL https://en.wikipedia.org/wiki/Graph_Query_Language

RPQ regular expressions as queries

I have wanted recursive queries for embedding egraphs in graph rewriting. share*  and E*

datalog could do it by materializing the needed transitive closure?
https://docs.google.com/document/d/1eq7jED7XMnOuq80PggkVKtqWTL1wQ8SdZMIWHSZLoPM/edit?tab=t.0

filtyer push down is smaller
hjow to use cost model

https://rustmagazine.org/issue-2/write-a-sql-optimizer-using-egg/

basic graph patterns

sqlite = bad?
It does suck encoding into it

paths asd first class

GPC - graph pattern calculus

cq conjunctive query
rpq - recursive path query
two way rpq

https://github.com/egraphs-good/egg/discussions/189

lexicographic path ordering

select(join(A,B)) -> join(select(A), select(B))
We make it bigger.

cross > join > select

associativity join(A, join(B,C,keys1),keys2) = join(join(A,B, keys3), C, keys4)

"proof relevant" associatvity?

commutavity join(A,B,keys) = join(B,A,keys)

Maybe join with no keys.
And rename nodes?
Just set expressions? Union, Inter, Select, 


https://arxiv.org/pdf/2202.10390

datalog is common 



column names as an analysis. rename anything not joined over before hand
the analysis informs the cost.


aurora? rl
single fixed point
why kat?
RPQ regular path query
rpql

fgh subsumes magic set
generating h is the task?

reachability vs all-pairs is the cnaonical 
projection followed by star vys projecting first and only have n vs n**2

https://rustmagazine.org/issue-2/write-a-sql-optimizer-using-egg/ ok yes. rows, cols size in their cost model are functions of c, the eclass.
it isn't really an analysis

Single nested loops. Just use var. Kind of reasonable?
1
loop(
foo(bar(var))
)




stefania is doing kat? So is that a useful model?

Optimized kleene algerba regex expresions
Model checking using graph databases

graph queries are binary kind of with property labels

Hmm. cloudpickle is actually kind of cool

oh colab has slide ashow



In [77]:
from egglog import *

class Math(Expr):
    def __init__(self, value: i64Like) -> None: ...
    @classmethod
    def var(cls, name: StringLike) -> "Math": ...
    def __add__(self, other: "Math") -> "Math": ...
    
egraph = EGraph()
x = Math.var("x")
dir(x + x)
type(get_callable_fn(x + x))

egglog.runtime.RuntimeFunction

In [ ]:
# https://github.com/egraphs-good/egglog-python/blob/a19aa868db875c619ea103fc954bcd5fba7466ae/python/egglog/examples/jointree.py
from egglog import *


class JoinTree(Expr):
    def __init__(self, name: StringLike) -> None: ...

    def join(self, other: JoinTree) -> JoinTree: ...

    @method(merge=lambda old, new: old.min(new))  # type:ignore[prop-decorator]
    @property
    def size(self) -> i64: ...


ra = JoinTree("a")
rb = JoinTree("b")
rc = JoinTree("c")
rd = JoinTree("d")
re = JoinTree("e")
rf = JoinTree("f")

query = ra.join(rb).join(rc).join(rd).join(re).join(rf)

egraph = EGraph()
egraph.register(
    set_(ra.size).to(50),
    set_(rb.size).to(200),
    set_(rc.size).to(10),
    set_(rd.size).to(123),
    set_(re.size).to(10000),
    set_(rf.size).to(1),
)


@egraph.register
def _rules(s: String, a: JoinTree, b: JoinTree, c: JoinTree, asize: i64, bsize: i64):
    # cost of relation is its size minus 1, since the string arg will have a cost of 1 as well
    yield rule(JoinTree(s).size == asize).then(set_cost(JoinTree(s), asize - 1))
    # cost/size of join is product of sizes
    yield rule(a.join(b), a.size == asize, b.size == bsize).then(
        set_(a.join(b).size).to(asize * bsize), set_cost(a.join(b), asize * bsize)
    )
    # associativity
    yield rewrite(a.join(b)).to(b.join(a))
    # commutativity
    yield rewrite(a.join(b).join(c)).to(a.join(b.join(c)))


egraph.register(query)
egraph.run(1000)
print(egraph.extract(query))
print(egraph.extract(query.size))

NameError: name 'JoinTree' is not defined

In [ ]:
from kdrag.all import *
import kdrag.theories.algebra.kleene as kl
import kdrag.solvers.egglog as egglog


In [46]:
kl.KLEENE[0].thm


ForAll([x, y, z], par(x, par(y, z)) == par(par(x, y), z))

In [47]:
from kdrag.all import *
import kdrag.theories.algebra.kleene as kl
import kdrag.solvers.egglog as egglog
kl.KLEENE

s = egglog.EgglogSolver()
s.add(kl.KLEENE)
s.let("t", kl.seq(kl.one,kl.zero))
s.run(10)
s.extract(kl.seq(kl.one,kl.zero))
#kl.KLEENE

[ExtractBest(<builtins.TermDag object at 0x7170a54ca330>, 1, TermApp('_0', []))]

In [48]:
s.cmds

['(sort Bool)',
 '(sort Kleene)',
 '(constructor _1 () Kleene)',
 '(constructor par (Kleene Kleene) Kleene)',
 '(constructor seq (Kleene Kleene) Kleene)',
 '(constructor star (Kleene) Kleene)',
 '(constructor _0 () Kleene)',
 '(rewrite (par X!284 (par Y!285 Z!286)) (par (par X!284 Y!285) Z!286))',
 '(rewrite (par X!287 Y!288) (par Y!288 X!287))',
 '(rewrite (par X!289 X!289) X!289)',
 '(rewrite (par X!290 (_0)) X!290)',
 '(rewrite (seq X!291 (seq Y!292 Z!293)) (seq (seq X!291 Y!292) Z!293))',
 '(rewrite (seq X!294 (_0)) (_0))',
 '(rewrite (seq X!295 (_1)) X!295)',
 '(rewrite (seq X!296 (par Y!297 Z!298)) (par (seq X!296 Y!297) (seq X!296 Z!298)))',
 '(rewrite (seq (par Y!300 Z!301) X!299) (par (seq Y!300 X!299) (seq Z!301 X!299)))',
 '(rewrite (star E!302) (par (_1) (seq E!302 (star E!302))))',
 '(rule ((= (par (par F!308 (seq E!307 X!306)) X!306) X!306)) ((union (par (seq (star E!307) F!308) X!306) X!306)))',
 '(let t (seq (_1) (_0)))',
 '(run 10)',
 '(extract (seq (_1) (_0)))']

In [8]:
p,q,r = smt.Bools("p q r")
smt.And(p,q)

import kdrag.theories.bool as bool_
s = egglog.EgglogSolver()

for a in [True, False]:
    for b in [True, False]:
        for e in [smt.And(a,b), smt.Or(a,b), smt.Not(a)]:
            s.add(e == smt.simplify(e))


EggSmolError: In 1:13-18: (true)
parse error: expected call expression

In [ ]:
%%file /tmp/sat.egg

(sort Bool)
(constructor True () Bool)
(constructor False () Bool)
(constructor And (Bool Bool) Bool)
(constructor Or (Bool Bool) Bool)
(constructor Not (Bool) Bool)
(union (And (True) (True)) (True))
(union (And (True) (False)) (False))
(union (And (False) (True)) (False))
(union (And (False) (False)) (False))
(union (Or (True) (True)) (True))
(union (Or (True) (False)) (True))
(union (Or (False) (True)) (True))
(union (Or (False) (False)) (False))
(union (Not (True)) (False))
(union (Not (False)) (True))






Overwriting /tmp/sat.egg


In [20]:
! egglog /tmp/sat.egg

[ERROR] In 19:1-28 of /tmp/sat.egg: (query (And (True) (False)))
    Unbound function query


should I propose what I think are interesting egglog features


In [ ]:
import lark
grammar = r"""
start: cmd*
cmd:   "(" "union" expr expr ")"   -> union
     | "(" "constructor" CNAME "(" sort* ")" sort ")"  -> constructor
     | "(" "sort" sort ")"         -> sort
     | "(" "rewrite" expr expr ")"     -> rewrite
     | "(" "rule" "(" expr* ")" "(" action* ")" ")"
     | "(" "run"  NUMBER ")"           -> run
     | "(" "let" CNAME expr ")"        -> let
    // | "(" "datatype" CNAME "(" constructor* ")" ")" -> datatyp  // loathsome datatype
?sort: CNAME
expr : "(" CNAME expr* ")"  -> app
// pat
action: "(" "union" expr expr ")"

%import common.WS 
%import common.CNAME
%import common.NUMBER
%ignore WS
"""

lark.Lark(grammar, start='start', parser='lalr')

class EgglogCmds(lark.Transformer):
    def __init__(self):
        self.cmds = []
    
    def union(self, items):
        self.cmds.append(('union', items[0], items[1]))
    
    def constructor(self, items):
        name = items[0]
        args = items[1:-1]
        ret_type = items[-1]
        self.cmds.append(('constructor', name, args, ret_type))
    
    def sort(self, items):
        name = items[0]
        self.cmds.append(('sort', name))
    
    def rewrite(self, items):
        self.cmds.append(('rewrite', items[0], items[1]))
    
    def app(self, items):
        name = items[0]
        args = items[1:]
        return (name, args)

    def rule(self, items):
        patterns = items[0:-1]
        actions = items[-1]
        self.cmds.append(('rule', patterns, actions))

    def action(self, items):
        return ('union', items[0], items[1])
def parse_egglog(code):
    parser = lark.Lark(grammar, start='start', parser='lalr')
    tree = parser.parse(code)
    transformer = EgglogCmds()
    transformer.transform(tree)
    return transformer.cmds

example = """
(sort Bool)
(constructor True () Bool)
(rewrite (And (True) (False)) (False))
(union (And (True) (True)) (True))
(rule ((And (p) (q))) ((union (And (p) (q)) (True))))
"""
parse_egglog(example)


[('sort', Token('CNAME', 'Bool')),
 ('constructor', Token('CNAME', 'True'), [], Token('CNAME', 'Bool')),
 ('rewrite',
  (Token('CNAME', 'And'),
   [(Token('CNAME', 'True'), []), (Token('CNAME', 'False'), [])]),
  (Token('CNAME', 'False'), [])),
 ('union',
  (Token('CNAME', 'And'),
   [(Token('CNAME', 'True'), []), (Token('CNAME', 'True'), [])]),
  (Token('CNAME', 'True'), []))]

In [9]:
s.cmds

['(sort Bool)', '(union (and (true) (true)) (true))']

In [ ]:
class App():
    f : str
    args : tuple["App", ...]
    kwargs : dict[str, "App"]




In [ ]:
from dataclasses import dataclass
class Rel(): ...

class Join():
    A : Rel
    B : Rel
    cols : list[str]

class Union():
    A : Rel
    B : Rel

class Star():
    R : Rel

class Proj(): # Select, Sigma
    R : Rel
    cond : ?


SyntaxError: invalid syntax (498765439.py, line 17)

In [ ]:
class Proj():
    cols : set[str]
    R : Rel
class Join():
    A : Rel
    B : Rel
    on : list[tuple[str, str]] # (A.col, B.col)
class Filter():
    vs : dict[str, any] # col -> value
    R : Rel



In [ ]:
@dataclass
class App():
    f : str
    args : tuple['App', ...] = ()
proj()

def rw():
    ...


In [ ]:

class GraphDB():
    vertices : set[object]
    edges: 


 rdflib https://rdflib.readthedocs.io/en/stable/gettingstarted/#a-sparql-query-example

In [ ]:
%%file /tmp/rel.egg

(datatype Query
    (Join Query Query) ; (Vec String)?
    (Union Query Query)
    (Star Query)
    ()
)



In [ ]:
import egglog
E = egglog.EGraph()

class Math(egglog.Expr):



In [3]:
dir(egglog)

['Action',
 'BackOff',
 'BaseExpr',
 'BigInt',
 'BigIntLike',
 'BigRat',
 'BigRatLike',
 'Bool',
 'BoolLike',
 'BuiltinExpr',
 'Command',
 'Container',
 'ConvertError',
 'CostModel',
 'EGraph',
 'EggSmolError',
 'Expr',
 'ExprCallable',
 'ExprValueError',
 'Fact',
 'GraphvizKwargs',
 'GreedyDagCost',
 'Map',
 'MapLike',
 'MultiSet',
 'Primitive',
 'PyObject',
 'Rational',
 'RewriteOrRule',
 'Ruleset',
 'Schedule',
 'Set',
 'SetLike',
 'StageInfo',
 'String',
 'StringLike',
 'TimeOnly',
 'Unit',
 'UnstableFn',
 'Vec',
 'VecLike',
 'WithPlan',
 '_BirewriteBuilder',
 '_EqBuilder',
 '_NeBuilder',
 '_RewriteBuilder',
 '_SetBuilder',
 '_UnionBuilder',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'back_off',
 'bindings',
 'birewrite',
 'builtins',
 'check',
 'check_eq',
 'config',
 'constant',
 'conversion',
 'convert',
 'converter',
 'declarations',
 'deconstruct',
 'default_cost_model',
 'define_expr_method',

In [1]:
%%file /tmp/query.ml


type constr = Eq of string * string

type query = Select of string * query | Join of query * query
  ;;


let opt1 (q : query) = match q with
  | Select (col, Join (q1, q2)) -> Some (col, q1, q2)
  | _ -> None
;;


print_endline "Hello, Pathquery!"

Writing /tmp/query.ml


In [2]:
! ocaml /tmp/query.ml

Hello, Pathquery!
